# Beautiful Soup Lab

### Introduction

In this lesson, we'll use beautiful soup and openai to develop a web scraper for the indeed website, storing the data in postgres.  We'll also use the SQLAlchemy and Flask libraries to use that data for a backend API.  Let's get started. 

### Getting Started

Let's begin by exploring the Indeed.com website.  In doing so, what we're looking for is the url we can make a request to, that we can ultimately scrape.

Ok, so go to Indeed.com, and then see how it works by typing in the job title `Data Engineer`, and a location of `New York, NY`, then click on `Find Jobs`.

<img src="data-eng-jobs.png" width="100%">

Finally, click on the second page of results.

<img src="./second-results.png" width="40%">

The key thing to really pay attention to is the url at the top as we navigate the website.  As we can see we have a url of `indeed.com/jobs` with various parameters.  

The `start=10` is a pagination parameter, which allows us to page page through results.  

> So here, we are not seeing the results at the very top, but from number 10 on, as we are on the second page and there are 10 results per page.

<img src="./indeed-url.png" width="80%">

Ok, so now it's time to write our first function.  Before doing so, first create a new python environment, and activate the environment.

Then install the necessary libraries for the project, which are listed in the `requirements.txt` file.

You can install these by running:

`pip3 install -r requirements.txt`

Then, you can run the tests for the `indeed_client` with the command:

```bash
python3 -m pytest tests/test_indeed_client.py
```

### Working with the Indeed client

Ok, so the first file we should work is the `indeed_client.py` file.  By client, we mean something that interacts directly with the external website -- `indeed.com`.

* `get_indeed_html` - `provided`

In that file, we wrote a function called `get_indeed_html` uses selenium to make a request to indeed.com.  It should automatically install the chrome driver, which you can see  more information about [here](https://www.selenium.dev/documentation/webdriver/getting_started/install_drivers/).



* `get_job_cards`
    * Now the `get_indeed_html` function returns the HTML from the entire page, so we now would like to write a function called `get_job_cards` calls our `get_indeed_html` function, and then selects the list of job cards on each page.  Notice that the relevant content appears to be located in the `td` items.
    * Pass the related test in the `test_indeed_client.py` file.

* `extract_text_from_card` - `provided`
    * Here the function, retrieves the text from a provided card.  We do some light clean up by using `strip` to remove whitespaces, and making sure we remove css.
    
* `get_id_from_card`
    * Each listed position also has an indeed id.  We would like that in addition to the text from the card.  Pass the related test.
    
    > Hint: For `get_id_from_card` look to the `a tag` nested inside of an individual `td`, and on that a tag, you can find the `data-jk` attribute that has the id.  In the image below, the id begins with `8cba`.

    > <img src="./data-jk-a.png" width="70%">

* `get_card_info`
    * This simply calls `extract_text_from_card` to pull out a list of the text elements, and then appends one additional string of 
    * `"job id: job_id"`
    * where `job_id` is the job_id we retrieved from the earlier method. 
    
* `get_card_infos`
    * This retrieves the card info (including the job_id) for each card in the html.

### Writing to a file

Ok, now so far we have retrieved some text.  Notice that this text is not perfectly clean.  But we don't need it to be perfectly clean -- we'll let openai interpret this text.  But first, we'll build some methods to write our text to a file.  From there, we'll have openai read this text.

Ok, so now for the methods on writing our text to a file.

* `build_text`
    * The first step is a method that will turn our `card_infos` list into some text.  So for each card_info list, add a `\n` to separate each element in a card info by a new line.  Then separate each card_info by two lines.  See the related test.
   * This is an example of how the card info texts should be formatted:
```
Data Engineer
HealthFirst
Staten Island, NY 10301
(
New Brighton area
)
Pay information not provided
Full-time
job id: af1c846c34ac0534

Data Engineer
NYPD Civilian Jobs
Manhattan, NY
job id: 9562c51b70acd54d
```

* `directory_name_builder`

Ok, so we are about to write this text to a file, but before we do we should build the `directory_name_builder` function.  This function generates the directory name that we will write the file to.   It takes 4 arguments, the `position`, `location`, `directory` and `date`.  So if there are arguments `directory_name_builder('data engineer', 'united states')` and the folder is built in the format of something like:

`..data/text_docs/data_engineer/nyc/2024-03-01`

So there is a directory should have a default argument of `..data/text_docs` and date has a default argument of `today`.  When that argument is today, the current date should be the inner most folder.  Notice that everything is lower case and there are no spaces in the folder name.

* `write_to_file` - `provided`
    * You can see that `write_to_file` uses the folder name generated from `directory_name_builder` and then creates this directory if it does not exist, and then writes the provided text to a file whose last character is the job index being scraped (remember that we can paginate through the jobs.

* `retrieve_and_write_pages` - `provided`
    * This will loop through a specified number of pages.  It has a step size because we want to pull 15 positions at a time.

Ok, so if you call `retrieve_and_write_pages` from the `main.py` or from the console, then you should see the relevant folder generated, and files generated with text of various positions inside of them.

## OpenAI Text to Json

Next, we can use openai's API to generate json from the data.  You can learn more about how to do that from [this resource](https://community.openai.com/t/how-do-i-use-the-new-json-mode/475890/11).  

This step is more prompt engineering than anything, and so we provided the code of our two methods for you.  

* `build_prompt`

This generates the prompt that we will provide to openai.  You can see the prompt here.
    * `prompt = f"""Format in json, the job_title, company_name, min_salary, max_salary, location, and presence as in-person, remote, hybrid or unknown of each of the jobs in the context.

    The json schema should include: 

    {JSON_SCHEMAS}

    Example:


    Senior Data Engineer
    Disney Entertainment & ESPN Technology
    New York, NY
    $136,038 - $182,490 a year
    job id: afece6001fb4eb54

    {ex_1}


    Context:

    {file_text}
    """   
`

So we tell openai what to do, and then we provide it a JSON schema of the output format it should generate.  You can see that in that schema we provide the key and for the value the datatype and a small description.
```python
JSON_SCHEMAS =  {
        "job_id": "string",
        "job_title": "string (do not include information about senority level, Good: data engineer, Bad: Senior data engineer)",
```

* One shot learning 

We then provide an example of an input and the output it should generate. 

```text
Example

Senior Data Engineer
Disney Entertainment & ESPN Technology
New York, NY
$136,038 - $182,490 a year
job id: afece6001fb4eb54

ex_1 = {'job_id': 'afece6001fb4eb54', 'job_title': 'Data Engineer',
    'company_name': 'Disney Entertainment & ESPN Technology',
    'seniority_level': 'senior', 'min_salary': 136038,
    'max_salary': 182490, 'city': 'New York',
        'state': 'NY', 'zipcode': None, 'presence': 'unknown'}
```

So this can help the AI model learn what it should produce.  If we provide no example it's called **zero shot learning**, one example is **one-shot learning**, and then two examples is **two-shot learning**.  No one has ever tried three.

Then after this example, we provide it the text from the file (the list of positions in a similar format to our example), and hopefully it will generate the json.

* `return_json_from(prompt)`

Ok, so this takes the text from the prompt, and feeds it to the model.  For the output to be json data, only specific openai models can be used, and we use `"gpt-3.5-turbo-1106"`.  Notice our response format: `response_format={ "type": "json_object" }`.

Finally it outputs a string, so we use `json_response = json.loads(json_content)` to turn that string into a list of dictionaries one for each position in our text.

# FileReader 

Ok, so now remember what we have built so far.

1. We now have code that scrapes positions from the indeed website, and writes them to the specified file.
2. We have code that can take text in the format of our file, and use openai to return json.


So the next step is to read the text from that file and from there we can use our `json_builder` to properly format it into json.

* `file_to_df(file_name)`

Given a file name, it should return the jobs in the file as a dataframe.  However, if the file has fewer than 20 characters, it should just return an empty dataframe.

* To build this, use the functions in our `json_builder` file.
* Before returning the dataframe, replace any values of `nan`, `''` or `unknown` in the dataframe with `None` -- this way when we ultimately persist this data in a database, it will be saved as null.

* `parse_from_file_name`
    * Now we're close to loading this data into a database.  However, beyond just loading each position into the database, we'll also want to associate that position with the scraping -- that is the html and date pulling the data.  Luckily, that information is encoded in the folder structure.
    So write a function called `parse_from_file_name` that a file like: 
        `text_docs/data_engineer/united_states/2024-03-01/results_2.txt`, will return a dictionary of each of the attributes (`position`, `location`, `date` and `job_idx`).

### Developing the Models

So now we have written code to scrape our html, save the text, and then extract data both from the text file, and from the *name* of the text file.  Next up is to save this data to a database.

To do this we'll need to code for two models: scraping and position, where a scraping has many positions.  We'll use flask_sqlalchemy to do this.  This requires a bit of setup.  

> **Hint** You can get a sense of how we set this up by referencing [this repository](https://github.com/apis-jigsaw/sql-alchemy-relations/tree/main/src) and [this lesson](https://colab.research.google.com/github/apis-jigsaw/sql-alchemy-relations/blob/main/lesson/index.ipynb).

0. Create the database, and specify the name of the environment in the `.env` file.  In the settings file, set that db connection string to a variable.  The name of my connection string is: `postgresql://localhost:5432/indeed_llm_scraper`.  You also will need to set up a test database, run the migrations against the development and test databases, and store the connection to the test database as an environmental variable.  You can look at the `settings.py` file to see how we use these variables.

1. `.flaskenv` - specify that we'll be using `server.py` as the location of the `FLASK_APP`.
2. `app/__init__.py` Here, we'll write the `create_app` function.  This should take the `db_conn` string as an argument.  
3. `server.py`
    * This is where we'll ultimately create the app and passing through the `db_conn` string.

Ok, once the setup is complete, the models can be written.

1. Position
2. Scraping 

You can see the underlying columns for the positions and scrapings models in the `migrations/create_tables.sql` folder.  Don't forget to also add the relationships so that a scraping has many positions.  Get the corresponding tests in the `tests/models` folder to pass.

### Back to FileReader

Ok, so now that we have our models set up, we can move beyond just reading our text file and returning a dataframe, to actually saving the data in our file to a database.

* `file_to_db(file_name, db_conn)`

This takes both a `file_name`, and our database connection string.  Given the `file_name`, it creates (and saves) a scraping object, and it also creates each of the positions in the file, and relates them to the scraping.  

Notice that to make sure that we have our sqlalchemy connection getting set up from our app before creating or saving any model instances, we'll need to write this code inside of an `app_context()`.  We provided some of this initial code for you.

Pass the related test.

**Then**, to pass the next test, you need to make sure that we do not duplicate our positions.  For this use the `job_id` to make sure when we call the `file_to_db` method we first check to make sure a position is not already in the database before inserting it.  Finally, have the return value be a list of positions that were added to the database (and only those positions).


* `list_files` - `provided`

* `files_to_db(dir_name)`

Next, now that we are able to load one file into the database, it's time to set up our code so we can a load all files in a folder to a database.  Run the `dir_to_db` method and pass the related test.
    

* Testing it out 

Now, if you run the `main.py` file, you can see the pipeline that we built in action.  
The `retrieve_and_write_pages` function will use our scraper to pull data from Indeed and write the text from each scraped position to a file.  It returns a list of file_names where the jobs are written to.  Then, the `files_to_db` function will take those files, turn the text to JSON (using openai) and then turn the json to scraping and position instances which are written to the database. 

### Putting it together

* Building out an API

Next, in the `create_app` file, we should also add an endpoint to our flask application called `positions` so that when we make a request to `/positions`, we should see the list of positions.  One issue is that the sqlalchemy instance returns extra attributes, in addition to our column attributes.  So add a `to_dict` method in the `Position` class.

```python
def to_dict(self):
    return dict((col, getattr(self, col)) for col in self.__table__.columns.keys())
```

Use this method when returning json of each of the positions in the `/positions` route.

We should get back something like the following:

> <img src="./returned-json.png">

### Scheduling with Prefect

Ok, so now that we have the code to pull data from indeed and write it to a database, the next step is to connect this to [this lesson](https://github.com/data-engineering-jigsaw/prefect-deployments/blob/main/5-prefect-deployments/code/index.py) or some of the [related documentation](https://docs.prefect.io/latest/concepts/schedules/).

We have provided a `pipeline.py` file for you.

Begin by creating two tasks:

* `retrieve_and_write_pages(position, location, pages)`
    * This calls the retrieve and write pages function in our indeed scraper, and should return the list of file names.
    
* `read_files_to_db(file_names)`
    * This reads the files to a database. 


From there create the following flow:

* `scrape_and_load_positions(positions, locations, pages)`
    * This takes a list of positions, locations and the number of pages to scrape from each search.
    * It should scrape and save each combination of positions and locations.
    
Adding a deployment to run it on a schedule.

Finally, we would want something like this to be run daily.  So let's test it out by adding a deployment that scrapes data engineer, analytics engineer positions for the cities of new york city and san francisco every 100 seconds.

Remember that you should be able to view the logs of this in `prefect.io`.

### Connecting to our dashboard

We already built a little dashboard of the scraped positions for you.  You can see the related code in the `frontend` folder.  To connect this to your backend do the following:
    
1. Call `flask run` to boot up your api

2. In a new tab, navigate to the `frontend` folder, and then run `streamlit run index.py`.

### Summary

In this lesson we used BeautifulSoup to parse down some initial text, OpenAI to process that text into JSON and SQLAlchemy to save and retrieve that data in a database.  From there, connected a flask API to our database, and developed a frontend dashboard that makes requests to the API.

We also set up a prefect pipeline that regularly pulls our data.